In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import statsmodels.api as sm

# Table 1

In [ ]:
magnesium = pd.read_excel(r"D:\주성\documents\2025\Mg comma\clinical features2.xlsx", sheet_name='Table2')
magnesium1 = magnesium.loc[magnesium['Mg Protocol Type']==1]
magnesium2 = magnesium.loc[magnesium['Mg Protocol Type']==2]
print(magnesium1.describe())
print(magnesium2.describe())

from scipy.stats import mannwhitneyu as mann
for col in ['Mg Starting Date', 'Mg Infusion Day', 'Loading Mg', 'Maintenance Mg', 'Total Mg', 'Max concentration']:
    s, p = mann(magnesium1[col], magnesium2[col])
    print(col, s, p)

# Figure 2

In [ ]:
df = pd.read_csv(r"E:\주성\documents\2025\Mg comma\data2.csv")

# CSV 파일 읽기
data = df

# 밴드별로 데이터 나누기
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']

plt.figure(figsize=(15, 10))
for idx, band in enumerate(bands, 1):
    plt.subplot(2, 3, idx)
    
    # 해당 밴드의 데이터만 추출
    band_data = data[data['band'] == band].copy()
    
    # 변화율 계산 ((post - pre) / pre)
    change = (band_data['mean_post_infusion'] - band_data['mean_pre_infusion']) / band_data['mean_pre_infusion']
    
    # 오름차순 정렬
    sorted_change = np.sort(change)
    
    # 바 차트 그리기
    plt.bar(range(len(sorted_change)), sorted_change)
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    plt.title(f'{band.capitalize()} Band')
    plt.ylim(-1, 2)  # y축 범위 수정

plt.suptitle('Power Change by Frequency Band', fontsize=14, y=1.02)    
plt.tight_layout()
plt.show()

# Table 3

비율 만드는 코드

In [3]:
feature.columns

Index(['Patient No.', 'Sex', 'Age', 'Class_SRSE', 'Class_pSRSE', 'Class_NORSE',
       'Coma', 'CIVAD Medications', 'Coma_Midazolam', 'Coma_Pentobarbital',
       'Coma_Propofol', 'Coma_Ketamine', 'Sz Type', 'Sz_Type_NCSE',
       'Sz_Type_FocalSE', 'Sz_Type_GCSE', 'Etiology', 'Etiology_Tumor',
       'Etiology_Drug', 'Etiology_Alcohol', 'Etiology_Infection',
       'Etiology_Metabolic', 'Etioloigy_Cryptogenic',
       'Etiology_Acute_vascular', 'Etiology_Autoimmune',
       'Etiolkogy_Remote_vascular', 'STESS', 'mSTESS_total', 'EMSE_4strata',
       'EMSE_6strata', 'ImmunoTx', 'Immunotherapy Medications',
       'ImmunoTx_number', 'ImmunoTx_Steroid', 'ImmunoTx_IVIG',
       'ImmunoTx_Rituximab', 'ImmunoTx_Tocilizumab', 'AED_number',
       'Seizure_days', 'Seizure_not_terminated', 'Hospital_days',
       'mRS_premorbid', 'Inhospital_Mortality', 'mRS_Discharge',
       'mRS_3mo_text', 'Szfree_at3mo', 'Recurred_SE_at3mo', 'Mg type',
       'time_until_Mg', 'total_Mg_date', 'Mg_loading_d

In [53]:
# clinical feature
feature = pd.read_excel(r"D:\주성\documents\2025\Mg comma\clinical features4.xlsx", sheet_name = 'Sheet1')
feature = feature.iloc[:-2, :]

# 파생변수 
feature['Immunotherapy'] = ~feature['Immunotherapy Medications'].isna()
feature['CIVADs'] = ~feature['CIVAD Medications'].isna()
feature['STESS_5'] = feature['STESS'] >= 5
feature['mRS_4'] = feature['mRS_premorbid'] >= 4
feature['CIVADs_num'] = feature['CIVAD Medications'].apply(lambda x: sum(1 for c in str(x) if c.isupper()) if pd.notna(x) else 0)
feature['Age_65'] = feature['Age'] >=65
feature['Mg_change'] = feature['max_Mg_conc'] - feature['Mg_baseline']
feature['Mg_8.5'] = feature['max_Mg_conc'] >= 8.5

feature.loc[feature['response']==True]

# Total subject
total_subject = feature['response'].sum()
print("total response subject:", total_subject)

# 범주형
feature_cat = feature[['Class_SRSE', 'CIVADs', 'Sex', 'Immunotherapy',
    'Class_NORSE', 'Mg type', 'STESS_5', 'mRS_4', 'Age_65', 'Etiology', 
    'Sz Type', 'Mg_8.5'
]]

for col in feature_cat.columns:
    cross = pd.crosstab(feature_cat[col], feature['response'])
    print(cross)

# 연속형
feature_num = feature[['CIVADs_num', 'AED_number', 'Age', 'STESS',  'mRS_premorbid', 'max_Mg_conc', 'Mg_change', 'response']]
print("responder", "\n", feature_num.loc[feature_num['response']==1].describe())
print("non-responder", "\n", feature_num.loc[feature_num['response']==0].describe())


total response subject: 13.0
response    0.0  1.0
Class_SRSE          
0.0           4    3
1.0           3   10
response  0.0  1.0
CIVADs            
False       4    3
True        3   10
response  0.0  1.0
Sex               
F           5    5
M           2    8
response       0.0  1.0
Immunotherapy          
False            3    5
True             4    8
response     0.0  1.0
Class_NORSE          
0.0            3    6
1.0            4    7
response  0.0  1.0
Mg type           
1.0         4   10
2.0         3    3
response  0.0  1.0
STESS_5           
False       2   10
True        5    3
response  0.0  1.0
mRS_4             
False       4   12
True        3    1
response  0.0  1.0
Age_65            
False       2    9
True        5    4
response                 0.0  1.0
Etiology                         
Acute vascular disease     1    0
Autoimmune                 2    4
Cryptogenic                3    3
Infectious encephalitis    0    1
Metabolic                  0    3
Remote va

통계값 구하기

In [56]:
# clinical feature
feature = pd.read_excel(r"D:\주성\documents\2025\Mg comma\clinical features4.xlsx", sheet_name = 'Sheet1')
feature = feature.iloc[:-2, :]

# 파생변수 
feature['Immunotherapy'] = ~feature['Immunotherapy Medications'].isna()
feature['CIVADs'] = ~feature['CIVAD Medications'].isna()
feature['STESS_5'] = feature['STESS'] >= 5
feature['mRS_4'] = feature['mRS_premorbid'] >= 4
feature['CIVADs_num'] = feature['CIVAD Medications'].apply(lambda x: sum(1 for c in str(x) if c.isupper()) if pd.notna(x) else 0)
feature['Age_65'] = feature['Age'] >=65
feature['Mg_change'] = feature['max_Mg_conc'] - feature['Mg_baseline']
feature['Mg_8.5'] = feature['max_Mg_conc'] >= 8.5

# fisher exact test with 95% CI

def fisher(table, alpha=0.05):
    # Fisher’s Exact Test 수행
    odds_ratio, p_value = stats.fisher_exact(table)
    return p_value 

# chi square test
def chisquare (table, alpha=0.05):
    chi, p_value, dof, exp = stats.chi2_contingency(table)
    return p_value

feature_cat = feature[['Class_SRSE', 'CIVADs', 'Sex', 'Immunotherapy',
    'Class_NORSE', 'Mg type', 'STESS_5', 'mRS_4', 'Age_65', 'Mg_8.5'
]]

feature_mulvar = feature[['Etiology', 'Sz Type']]

feature_num = feature[['CIVADs_num', 'AED_number', 
'Age', 'STESS',  'mRS_premorbid', 'max_Mg_conc', 'Mg_change']]

results = {}
for col in feature_cat.columns:
    table = pd.crosstab(feature_cat[col], feature['response'])
    p_value = fisher(table)
    results[col] = {'p-value': p_value}

for col in feature_mulvar.columns:
    table = pd.crosstab(feature_mulvar[col], feature['response'])
    p_value = chisquare(table)
    results[col] = {'p-value': p_value}

for col in feature_num.columns:
    nan_indices = feature_num[col][feature_num[col].isna()].index
    X = feature_num[col].drop(nan_indices)
    X = sm.add_constant(X)
    y = feature['response'].drop(nan_indices)
    model = sm.Logit(y, X)
    result = model.fit()
    p_value = result.pvalues[1]
    results[col] = {'p-value': p_value}

result_df = pd.DataFrame(results).T

Optimization terminated successfully.
         Current function value: 0.584098
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.634731
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595755
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.602469
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601632
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.647052
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.620300
         Iterations 5


C:\Users\Brain_Science\AppData\Local\Temp\ipykernel_27008\1029846363.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = result.pvalues[1]
C:\Users\Brain_Science\AppData\Local\Temp\ipykernel_27008\1029846363.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = result.pvalues[1]
C:\Users\Brain_Science\AppData\Local\Temp\ipykernel_27008\1029846363.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = resu

In [57]:
result_df

,p-value
Class_SRSE,0.173581
CIVADs,0.173581
Sex,0.349845
Immunotherapy,1.000000
Class_NORSE,1.000000
Mg type,0.612616
STESS_5,0.062332
mRS_4,0.101135
Age_65,0.159675
Mg_8.5,1.000000
